In [7]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import pymongo
import time
import random
import pandas as pd

In [2]:
# mongoDB 연결 and crawling_review collection 생성
conn = pymongo.MongoClient()
db = conn.crawling_review

In [4]:
# 네이버 평점 url, 여러 페이지 조회하기 위해 url page= 부분 format
base_url = "https://movie.naver.com/movie/point/af/list.naver?&page={}"

data_list = []

In [5]:
# 평점 크롤링 
for page in range(1,1000):
    url = base_url.format(page)
    res = requests.get(url)
    # 200 : 요청을 제대로 처리햇다는 뜻
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        html_path = soup.select('td.title')
        for td in html_path:
            # .text로 조회하면 태그들도 포함되므로 br태그 그 태그 내용 찾기위해 next_sibling
            review = td.select_one('br').next_sibling.text.strip()
            star = td.select_one('div.list_netizen_score > em').text.strip()
            data_list.append((star,review))
        delay = round(random.uniform(0.2,1.2),2)
        time.sleep(delay)
print('완료')

완료


In [42]:
star = data['평점'].values
star = list(star)
comment = data['리뷰'].values
comment = list(comment)

In [43]:
# mongoDB 저장
doc = {
    'star':star,
    'comment':comment
}
db.movies.insert_one(doc)

In [44]:
# csv 저장
data

,index,평점,리뷰
0,0,10,평점이 낮아서 안보려다 봤는데 보길 정말 잘했슴ㄷㅏ.저는 지루하지않았고 후반부로 갈...
1,1,10,가슴 시리도록 기억에 오래 남을 영화. 공유 전도연의 연기는.. 잊혀지지 않을 것 ...
2,2,1,대체... 이게 뭐죠?
3,3,9,우리가 자라고 우리가 사랑을 하고 우리가 결혼을 하고 우리가 아이를 키우고 우리가 ...
4,4,1,보는 내가 유체이탈할뻔 보다가 나온 영화는 처음임답답하고 주인공 본인도 몰라 답답한...
...,...,...,...
9985,9985,6,님의 하드캐리
9986,9986,7,로맨스 영화를 가장한 SF물이라서 뭔가 아쉬움이 남는 영화.개봉당시 포스터만 봐도 ...
9987,9987,10,"연출, 연기 다 좋습니다"
9988,9988,6,아저씨+뷰티인사이드
